# ETL: Raw → Silver

Transformação e limpeza dos dados brutos (Raw) para a camada Silver da arquitetura Medallion.

**Principais transformações:**
- Padronização de nomes de colunas
- Remoção de duplicados e dados redundantes
- Validação de qualidade dos dados numéricos
- Tratamento de valores ausentes
- Conversão de tipos de dados

In [1]:
# Bibliotecas necessárias
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
# Leitura do CSV original
csv_path = os.path.join('Data Layer', 'raw', 'ncr_ride_bookings.csv')

if not os.path.exists(csv_path):
    csv_path = "/kaggle/input/uber-analise/ncr_ride_bookings.csv"
    if not os.path.exists(csv_path):
        csv_path = os.path.join('..', 'Data Layer', 'raw', 'ncr_ride_bookings.csv')

print(f"Carregando dados de: {csv_path}")
df_bronze = pd.read_csv(csv_path)

print(f"Total de registros carregados: {len(df_bronze):,}")
df_bronze.head()

Carregando dados de: ../Data Layer/raw/ncr_ride_bookings.csv
Total de registros carregados: 150,000
Total de registros carregados: 150,000


,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,...,Reason for cancelling by Customer,Cancelled Rides by Driver,Driver Cancellation Reason,Incomplete Rides,Incomplete Rides Reason,Booking Value,Ride Distance,Driver Ratings,Customer Rating,Payment Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


## Nome das Colunas

Padronização dos nomes das colunas para garantir consistência e compatibilidade com o banco de dados.  
Essa etapa ajusta os nomes para seguir o padrão definido no data warehouse (uso de snake_case, remoção de espaços e caracteres especiais).

In [3]:
# Cria uma cópia da camada Bronze
df_silver = df_bronze.copy()

# Renomeia colunas para o padrão unificado (Silver)
df_silver.rename(columns={
    'Booking ID': 'Booking_ID',
    'Booking Status': 'Booking_Status',
    'Customer ID': 'Customer_ID',
    'Vehicle Type': 'Vehicle_Type',
    'Pickup Location': 'Pickup_Location',
    'Drop Location': 'Drop_Location',
    'Avg VTAT': 'Avg_VTAT',
    'Avg CTAT': 'Avg_CTAT',
    'Reason for cancelling by Customer': 'Reason_for_cancelling_by_Customer',
    'Driver Cancellation Reason': 'Driver_Cancellation_Reason',
    'Incomplete Rides Reason': 'Incomplete_Rides_Reason',
    'Booking Value': 'Booking_Value',
    'Ride Distance': 'Ride_Distance',
    'Driver Ratings': 'Driver_Ratings',
    'Customer Rating': 'Customer_Rating',
    'Payment Method': 'Payment_Method'
}, inplace=True)

df_silver.head()


,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,Avg_VTAT,Avg_CTAT,...,Reason_for_cancelling_by_Customer,Cancelled Rides by Driver,Driver_Cancellation_Reason,Incomplete Rides,Incomplete_Rides_Reason,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating,Payment_Method
0,2024-03-23,12:29:38,"""CNR5884300""",No Driver Found,"""CID1982111""",eBike,Palam Vihar,Jhilmil,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-11-29,18:01:39,"""CNR1326809""",Incomplete,"""CID4604802""",Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.9,14.0,...,NaN,NaN,NaN,1.0,Vehicle Breakdown,237.0,5.73,NaN,NaN,UPI
2,2024-08-23,08:56:10,"""CNR8494506""",Completed,"""CID9202816""",Auto,Khandsa,Malviya Nagar,13.4,25.8,...,NaN,NaN,NaN,NaN,NaN,627.0,13.58,4.9,4.9,Debit Card
3,2024-10-21,17:17:25,"""CNR8906825""",Completed,"""CID2610914""",Premier Sedan,Central Secretariat,Inderlok,13.1,28.5,...,NaN,NaN,NaN,NaN,NaN,416.0,34.02,4.6,5.0,UPI
4,2024-09-16,22:08:00,"""CNR1950162""",Completed,"""CID9933542""",Bike,Ghitorni Village,Khan Market,5.3,19.6,...,NaN,NaN,NaN,NaN,NaN,737.0,48.21,4.1,4.3,UPI


## Remoção de Coluna Redundante

As colunas de flags apresentam informações que já estão representadas na variável Booking_status. Portanto, mantê-las seria redundante e poderia causar duplicidade semântica ou inconsistências nas análises.

In [4]:
# Remove colunas de flags irrelevantes
df_silver.drop(columns=[
    'Cancelled Rides by Customer', 
    'Cancelled Rides by Driver', 
    'Incomplete Rides'
], inplace=True, errors='ignore')

## Tratamento de Texto

Aplicando tratamento de texto nas colunas necessárias.

In [5]:
# Remove aspas desnecessárias
df_silver['Booking_ID'] = df_silver['Booking_ID'].astype(str).str.replace('"', '', regex=False)
df_silver['Customer_ID'] = df_silver['Customer_ID'].astype(str).str.replace('"', '', regex=False)

## Remoção de Tuplas com o Identificador Único Duplicado

Existe um erro no dataset que permite que algumas tuplas tenham o Booking_ID duplicado

In [6]:
# Remove dados que possuem a coluna 'Booking_ID' duplicada
initial_count = len(df_silver)
df_silver.drop_duplicates(subset=['Booking_ID'], keep='first', inplace=True)
final_count = len(df_silver)
print(f"Quantidade de dados duplicados: {initial_count - final_count}")

Quantidade de dados duplicados: 1233


## Remoção de Outliers Extremos

Remove apenas valores exorbitantes que são claramente erros de registro:
- **Booking Value**: Remove valores acima do percentil 99.9% (>3.434 INR)
- **Avg VTAT**: Análise mostrou que não há outliers problemáticos

In [7]:
# Remoção de outliers extremos
# Análise identificou valores exorbitantes que podem ser erros de registro

print("="*80)
print("REMOÇÃO DE OUTLIERS EXTREMOS")
print("="*80)

inicial = len(df_silver)

# Filtra apenas corridas completadas para análise
completed_mask = df_silver['Booking_Status'] == 'Completed'

# 1. Booking_Value: Remover valores acima do percentil 99.9
if 'Booking_Value' in df_silver.columns:
    p999_booking = df_silver.loc[completed_mask, 'Booking_Value'].quantile(0.999)
    outliers_booking = df_silver[completed_mask & (df_silver['Booking_Value'] > p999_booking)]
    
    print(f"\n BOOKING VALUE:")
    print(f"   Percentil 99.9%: {p999_booking:.2f} INR")
    print(f"   Outliers extremos identificados: {len(outliers_booking)}")
    print(f"   Valores entre {p999_booking:.2f} e {df_silver['Booking_Value'].max():.2f} INR")
    
    # Remove apenas outliers extremos de corridas completadas
    df_silver = df_silver[~((df_silver['Booking_Status'] == 'Completed') & (df_silver['Booking_Value'] > p999_booking))]
    
    print(f"    Removidos: {inicial - len(df_silver)} registros")

# 2. Avg_VTAT: Análise mostrou que não há outliers problemáticos (máx = 15.0)
print(f"\n AVG VTAT:")
print(f"   Máximo: {df_silver['Avg_VTAT'].max():.2f}")
print(f"    Sem outliers extremos - valores dentro do esperado")

final = len(df_silver)
print(f"\n RESUMO:")
print(f"   Registros inicial: {inicial:,}")
print(f"   Registros final: {final:,}")
print(f"   Removidos: {inicial - final:,} ({(inicial - final)/inicial*100:.2f}%)")
print("="*80)

REMOÇÃO DE OUTLIERS EXTREMOS

 BOOKING VALUE:
   Percentil 99.9%: 3436.26 INR
   Outliers extremos identificados: 93
   Valores entre 3436.26 e 4277.00 INR
    Removidos: 93 registros

 AVG VTAT:
   Máximo: 20.00
    Sem outliers extremos - valores dentro do esperado

 RESUMO:
   Registros inicial: 148,767
   Registros final: 148,674
   Removidos: 93 (0.06%)


## Tratamento de valores ausentes (NaN)

Esta etapa realiza o tratamento de dados nulos para garantir consistência e integridade do dataset Silver:

**Motivos de cancelamento:** substitui valores ausentes por "Reason Unknown".

**Colunas numéricas:** converte para tipo numérico seguro e substitui valores nulos pela média da coluna.

**Método de pagamento:** preenche valores ausentes com a moda (valor mais frequente).

In [8]:
# Substitui valores ausentes por texto padrão nas colunas de motivos
for col in ['Incomplete_Rides_Reason', 'Driver_Cancellation_Reason', 'Reason_for_cancelling_by_Customer']:
    if col in df_silver.columns:
        df_silver[col] = df_silver[col].fillna('Reason Unknown')

# Converte colunas numéricas para tipo numérico seguro
num_cols = ['Avg_VTAT', 'Avg_CTAT', 'Booking_Value', 'Ride_Distance', 'Driver_Ratings', 'Customer_Rating']
for col in num_cols:
    if col in df_silver.columns:
        df_silver[col] = pd.to_numeric(df_silver[col], errors='coerce')

# Imputa a média nas colunas numéricas
for col in num_cols:
    if col in df_silver.columns:
        df_silver[col] = df_silver[col].fillna(df_silver[col].mean())

# Imputa a moda na coluna de método de pagamento (se existir)
if 'Payment_Method' in df_silver.columns:
    mode_value = df_silver['Payment_Method'].mode(dropna=True)
    if not mode_value.empty:
        df_silver['Payment_Method'] = df_silver['Payment_Method'].fillna(mode_value[0])

In [9]:
# Visualiza uma amostra dos dados transformados
df_silver.head(10)

,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,Avg_VTAT,Avg_CTAT,Reason_for_cancelling_by_Customer,Driver_Cancellation_Reason,Incomplete_Rides_Reason,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating,Payment_Method
0,2024-03-23,12:29:38,CNR5884300,No Driver Found,CID1982111,eBike,Palam Vihar,Jhilmil,8.454867,29.149114,Reason Unknown,Reason Unknown,Reason Unknown,505.320918,24.63988,4.230746,4.404364,UPI
1,2024-11-29,18:01:39,CNR1326809,Incomplete,CID4604802,Go Sedan,Shastri Nagar,Gurgaon Sector 56,4.900000,14.000000,Reason Unknown,Reason Unknown,Vehicle Breakdown,237.000000,5.73000,4.230746,4.404364,UPI
2,2024-08-23,08:56:10,CNR8494506,Completed,CID9202816,Auto,Khandsa,Malviya Nagar,13.400000,25.800000,Reason Unknown,Reason Unknown,Reason Unknown,627.000000,13.58000,4.900000,4.900000,Debit Card
3,2024-10-21,17:17:25,CNR8906825,Completed,CID2610914,Premier Sedan,Central Secretariat,Inderlok,13.100000,28.500000,Reason Unknown,Reason Unknown,Reason Unknown,416.000000,34.02000,4.600000,5.000000,UPI
4,2024-09-16,22:08:00,CNR1950162,Completed,CID9933542,Bike,Ghitorni Village,Khan Market,5.300000,19.600000,Reason Unknown,Reason Unknown,Reason Unknown,737.000000,48.21000,4.100000,4.300000,UPI
5,2024-02-06,09:44:56,CNR4096693,Completed,CID4670564,Auto,AIIMS,Narsinghpur,5.100000,18.100000,Reason Unknown,Reason Unknown,Reason Unknown,316.000000,4.85000,4.100000,4.600000,UPI
6,2024-06-17,15:45:58,CNR2002539,Completed,CID6800553,Go Mini,Vaishali,Punjabi Bagh,7.100000,20.400000,Reason Unknown,Reason Unknown,Reason Unknown,640.000000,41.24000,4.000000,4.100000,UPI
7,2024-03-19,17:37:37,CNR6568000,Completed,CID8610436,Auto,Mayur Vihar,Cyber Hub,12.100000,16.500000,Reason Unknown,Reason Unknown,Reason Unknown,136.000000,6.56000,4.400000,4.200000,UPI
8,2024-09-14,12:49:09,CNR4510807,No Driver Found,CID7873618,Go Sedan,Noida Sector 62,Noida Sector 18,8.454867,29.149114,Reason Unknown,Reason Unknown,Reason Unknown,505.320918,24.63988,4.230746,4.404364,UPI
9,2024-12-16,19:06:48,CNR7721892,Incomplete,CID5214275,Auto,Rohini,Adarsh Nagar,6.100000,26.000000,Reason Unknown,Reason Unknown,Other Issue,135.000000,10.36000,4.230746,4.404364,Cash


## Exportação dos Dados Tratados

In [10]:
import psycopg2
from psycopg2 import extras
from dotenv import load_dotenv

# Verificar se df_silver está definido
if 'df_silver' not in locals():
    raise NameError("df_silver não está definido. Execute todas as células anteriores primeiro.")

# Remover colunas de flags de outliers se existirem (geradas em execuções anteriores)
outlier_cols = [col for col in df_silver.columns if '_is_outlier' in col]
if outlier_cols:
    df_silver = df_silver.drop(columns=outlier_cols)
    print(f"Removidas {len(outlier_cols)} colunas de flags antigas: {outlier_cols}\n")

print(f"Total de registros a serem salvos: {len(df_silver):,}")

# Caminho para salvar a camada Silver
SILVER_PATH = os.path.join('../Data Layer', 'silver', 'uber_silver.csv')
os.makedirs(os.path.dirname(SILVER_PATH), exist_ok=True)

df_silver.to_csv(SILVER_PATH, index=False)
print(f"Dados transformados salvos em '{SILVER_PATH}'.")

# Função para obter string de conexão (usa .env se disponível)
def get_db_connection_info():
    load_dotenv()
    url = os.getenv('DB_URL')
    db_env = os.getenv('DB_ENV')
    if url is not None and db_env == 'prod':
        return url

    DB_USER = os.getenv('POSTGRES_USER', 'postgres')
    DB_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'postgres')
    DB_HOST = os.getenv('POSTGRES_HOST', 'localhost')
    DB_PORT = os.getenv('POSTGRES_PORT', '5432')
    DB_NAME = os.getenv('POSTGRES_NAME', 'uber')

    return {
        'host': DB_HOST,
        'dbname': DB_NAME,
        'user': DB_USER,
        'password': DB_PASSWORD,
        'port': DB_PORT
    }

# Carregar DDL e inserir
# Tentar múltiplos caminhos para o arquivo DDL
DDL_FILE = os.path.join('Data Layer', 'silver', 'silver_ddl.sql')
if not os.path.exists(DDL_FILE):
    DDL_FILE = os.path.join('..', 'Data Layer', 'silver', 'silver_ddl.sql')
if not os.path.exists(DDL_FILE):
    print(f"AVISO: Arquivo DDL não encontrado em: {DDL_FILE}")
    print("Tentando conectar sem executar DDL...")

conn_info = get_db_connection_info()

try:
    print('\nIniciando carregamento na camada SILVER (Postgres)...')
    df_to_load = df_silver.copy()

    # Ajustes de tipos: garantir colunas coerentes
    if 'Date' in df_to_load.columns:
        df_to_load['Date'] = pd.to_datetime(df_to_load['Date'], errors='coerce').dt.date
    if 'Time' in df_to_load.columns:
        # Converter Time para string formato HH:MM:SS, mantendo compatibilidade com PostgreSQL
        # Primeiro converte para datetime, depois para string
        time_col = pd.to_datetime(df_to_load['Time'], errors='coerce', format='%H:%M:%S')
        # Aplicar strftime apenas nos valores não-nulos
        df_to_load['Time'] = time_col.apply(lambda x: x.strftime('%H:%M:%S') if pd.notna(x) else None)

    # Conectar ao banco
    conn = psycopg2.connect(**conn_info)
    cur = conn.cursor()
    
    # Lê e executa DDL se o arquivo existir
    if os.path.exists(DDL_FILE):
        print(f'Executando DDL de: {DDL_FILE}')
        
        # Dropar tabela existente para garantir estrutura correta
        try:
            cur.execute('DROP TABLE IF EXISTS silver.uber_silver CASCADE')
            conn.commit()
            print('Tabela anterior removida.')
        except Exception as drop_error:
            print(f'Aviso ao dropar tabela: {drop_error}')
        
        # Criar schema e tabela
        with open(DDL_FILE, 'r', encoding='utf-8') as f:
            ddl_sql = f.read()
        cur.execute(ddl_sql)
        conn.commit()
        print('Tabela criada com sucesso.')
    else:
        print('Arquivo DDL não encontrado. Assumindo que schema e tabela já existem.')


    # Converter nomes das colunas para lowercase (compatível com DDL)
    df_to_load.columns = df_to_load.columns.str.lower()
    
    # Inserção em batches
    table = 'silver.uber_silver'
    cols = list(df_to_load.columns)
    cols = [c for c in cols if c != 'index']
    insert_query = f"INSERT INTO {table} ({', '.join(cols)}) VALUES ({', '.join(['%s'] * len(cols))})"

    records = [tuple(None if (pd.isna(v)) else v for v in row) for row in df_to_load[cols].itertuples(index=False, name=None)]
    extras.execute_batch(cur, insert_query, records, page_size=1000)
    conn.commit()

    print(f'Carga concluída: {len(records)} linhas inseridas em {table}.')

except Exception as e:
    print('Erro durante carregamento na camada SILVER:', e)
    if 'conn' in locals():
        conn.rollback()
finally:
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()

print('\nProcesso ETL Raw -> Silver concluído!')

Total de registros a serem salvos: 148,674
Dados transformados salvos em '../Data Layer/silver/uber_silver.csv'.

Iniciando carregamento na camada SILVER (Postgres)...
Dados transformados salvos em '../Data Layer/silver/uber_silver.csv'.

Iniciando carregamento na camada SILVER (Postgres)...
Executando DDL de: ../Data Layer/silver/silver_ddl.sql
Tabela anterior removida.
Tabela criada com sucesso.
Executando DDL de: ../Data Layer/silver/silver_ddl.sql
Tabela anterior removida.
Tabela criada com sucesso.
Carga concluída: 148674 linhas inseridas em silver.uber_silver.

Processo ETL Raw -> Silver concluído!
Carga concluída: 148674 linhas inseridas em silver.uber_silver.

Processo ETL Raw -> Silver concluído!
